In [1]:
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import time
from scipy import optimize
%matplotlib inline

# Newton's method with quality constraints

First we implement an extension of Newton’s method to include equality
constraints. We will need it later for building an interior-point barrier method. The implementation follows chapter 10 of Boyd Vandenberghe Convex optimization 2004 book.

In [2]:


class NewtonMethodWithEqualityConstraints:
    def __init__(self):
        pass
    
    def _get_newton_step(self, grad_func, hessian_func, A, x):
        lhs_matrix = np.zeros(shape=(len(x) + A.shape[0], len(x) + A.shape[0]), dtype=x.dtype)
        # print('DEBUG: mat shape {}'.format(lhs_matrix.shape))

        lhs_matrix[:len(x), :len(x)] = hessian_func(x)
        lhs_matrix[len(x):, :len(x)] = A.copy()
        lhs_matrix[:len(x), len(x):] = A.T.copy()
        
        rhs_vec = np.zeros(len(x) + A.shape[0], dtype=x.dtype)
        rhs_vec[:len(x)] = -grad_func(x)
        
        # print('DEBUG: mat shape {} ; vec shape {}'.format(lhs_matrix.shape, rhs_vec.shape))
        
        solution = np.linalg.solve(lhs_matrix, rhs_vec)
        return solution[:len(x)]
    
    def _get_newton_decrement(self, hessian_func, x, newton_step):
        return np.sqrt(newton_step.T @ hessian_func(x) @ newton_step)
    
    def _backtrack_step_size(self, func, grad_func, x, step, c=0.1, tau=0.1):
        m = step.T @ grad_func(x)
        step_size = 1.
        while func(x) - func(x + step_size * step) < step_size * (-c*m):
            # print('DEBUG: x={}; x + step_size * step={}; func(x)={}; '.format(lhs_matrix.shape))
            step_size *= tau
        return step_size
        
    def minimize(self, func, grad_func, hessian_func, A, b, x0, tol=1e-6):
        while True:
            step = self._get_newton_step(grad_func=grad_func,
                                         hessian_func=hessian_func,
                                         A=A, x=x0)
            decrement = self._get_newton_decrement(hessian_func=hessian_func,
                                                   x=x0, newton_step=step)
            if decrement**2 / 2. < tol:
                break
            
            step_size = self._backtrack_step_size(func=func,
                                                  grad_func=grad_func,
                                                  x=x0, step=step)
            x0 = x0 + step_size * step
        return x0

Let's test our implementation of Newton's method with solving a simple optimization problem defined as follows:

minimize $f(x, y) = x^2 + y^2$

subject to $x = y + 1$.

We expect to see optimal solution with $x = \frac{1}{2}$ and $y = -\frac{1}{2}$.

In [3]:
f = lambda x: np.sum(x**2)
df = lambda x: 2*x
d2f = lambda x: np.diag([2., 2.])

solver = NewtonMethodWithEqualityConstraints()

A = np.array([1., -1.]).reshape(1, 2)
b = np.array([1.])
x0 = np.array([11., 10.])

solver.minimize(func=f, grad_func=df, hessian_func=d2f, A=A, b=b, x0=x0)

array([ 0.5, -0.5])

# Interior-point barrier method
Now we are ready to implement the barrier method. The implementations follows chapter 11 of Boyd Vandenberghe Convex optimization 2004 book.

In [4]:
class BarrierSolver:
    """An implemetation of barrier method: an interior-point algorithm 
    from Boyd Vandenberghe Convex optimization 2004"""
    def __init__(self, func0, grad_func0, hessian_func0, barrier_func,
                       grad_barrier_func, hessian_barrier_func,
                       A, b, m):
        self._f0 = func0
        self._gf0 = grad_func0
        self._hf0 = hessian_func0
        
        self._bf = barrier_func
        self._gbf = grad_barrier_func
        self._hbf = hessian_barrier_func
        
        self._A = A
        self._b = b
        self._m = m
        
        self._newtons_method = NewtonMethodWithEqualityConstraints()
    
    def _get_functions(self, t):
        func = lambda x: t * self._f0(x) + self._bf(x)
        grad_func = lambda x: t * self._gf0(x) + self._gbf(x)
        hessian_func = lambda x: t * self._hf0(x) + self._hbf(x)
        return func, grad_func, hessian_func
    
    def _centering_step(self, x, t):
        f, gf, hf = self._get_functions(t)
        x_centered = self._newtons_method.minimize(func=f,
                                                   grad_func=gf,
                                                   hessian_func=hf,
                                                   A=self._A,
                                                   b=self._b,
                                                   x0=x)
        return x_centered
    
    def minimize(self, x0, t=1., mu=10., tol=1e-6):
        num_iter = 0
        print('num_iter={}; x={}'.format(num_iter, x0))
        while True:
            x_centered = self._centering_step(x0, t)
            x0 = x_centered
            num_iter += 1
            print('num_iter={}; x={}'.format(num_iter, x0))
            if self._m / t < tol:
                break
            t *= mu
        return x0

Now we apply the implemented method to the problem of communication channel capacity. 

In [5]:
def func(z, c, n, m):
    """ z = (x1, ..., xn, y1, ..., ym)"""
    assert len(z) == n + m
    assert len(c) == n
    
    comp1 = np.dot(c, z[:n])
    comp2 = np.sum([y_j * np.log2(y_j) for y_j in z[-m:]])
    return comp1 + comp2

def grad_func(z, c, n, m):
    """ z = (x1, ..., xn, y1, ..., ym)"""
    assert len(z) == n + m
    assert len(c) == n

    res = np.zeros(len(z), dtype=z.dtype)
    res[:n] = c
    res[-m:] = 1. / np.log(2.) + np.log2(z[-m:])
    return res

def hessian_func(z, c, n, m):
    """ z = (x1, ..., xn, y1, ..., ym)"""
    assert len(z) == n + m
    assert len(c) == n
    
    res = np.zeros(shape=(n + m, n + m), dtype=z.dtype)
    for j in range(m):
        res[n + j, n + j] = 1. / (z[n + j] * np.log(2.))
    return res

def barrier_func(z, n, m):
    """ z = (x1, ..., xn, y1, ..., ym)"""
    assert len(z) == n + m
    
    if np.min(z[:n]) < 0:
        return np.inf
    return -np.sum([np.log(x_i) for x_i in z[:n]])

def grad_barrier_func(z, n, m):
    """ z = (x1, ..., xn, y1, ..., ym)"""
    assert len(z) == n + m
    
    res = z.copy()
    res = -1. / res
    res[-m:] = 0
    return res

def hessian_barrier_func(z, n, m):
    """ z = (x1, ..., xn, y1, ..., ym)"""
    assert len(z) == n + m
    
    res = np.zeros(shape=(n + m, n + m), dtype=z.dtype)
    for i in range(n):
        res[i, i] = 1. / z[i]**2
    return res

In [6]:
np.random.seed(420)

n = 4
m = 5
P = np.array([np.random.uniform(size=n) for x in np.zeros(m)])
P /= P.sum(axis=0)
c_t = np.array([-np.sum(x * np.log2(x)) for x in P.T])
print(P, P.shape)


A = np.zeros(shape=(1 + m, n + m), dtype=P.dtype)
A[0, :n] = 1.
A[1:, :n] = -P
A[1:, n:] = np.diag(np.ones(m, dtype=P.dtype))

b = np.zeros(m + 1, dtype=P.dtype)
b[0] = 1.


x0 = np.random.uniform(low=0, high=10, size=n)
x0 = x0 / np.sum(x0)

y0 = P @ x0

z0 = np.hstack([x0, y0])
z0

[[0.13141014 0.18987113 0.11846558 0.09559841]
 [0.36143627 0.26392752 0.15641988 0.05927288]
 [0.26076315 0.25090666 0.04091871 0.04989451]
 [0.09872321 0.09866078 0.34360467 0.03738794]
 [0.14766723 0.19663391 0.34059116 0.75784627]] (5, 4)


array([0.35261514, 0.10243534, 0.34480155, 0.20014797, 0.12576766,
       0.22028057, 0.14174586, 0.17087619, 0.34132972])

In [7]:
solver = BarrierSolver(func0=lambda z: func(z, c_t, n, m),
                       grad_func0=lambda z: grad_func(z, c_t, n, m),
                       hessian_func0=lambda z: hessian_func(z, c_t, n, m),
                       barrier_func=lambda z: barrier_func(z, n, m),
                       grad_barrier_func=lambda z: grad_barrier_func(z, n, m),
                       hessian_barrier_func=lambda z: hessian_barrier_func(z, n, m),
                       A=A, b=b, m=m)
sol = solver.minimize(z0)

num_iter=0; x=[0.35261514 0.10243534 0.34480155 0.20014797 0.12576766 0.22028057
 0.14174586 0.17087619 0.34132972]
num_iter=1; x=[0.24792197 0.24174085 0.24704049 0.26329669 0.13291561 0.20765845
 0.14854894 0.14305438 0.36782262]
num_iter=2; x=[0.25000206 0.19396497 0.22186723 0.33416574 0.1279105  0.19606392
 0.13960998 0.13254613 0.40386948]
num_iter=3; x=[0.34874894 0.0693509  0.16695533 0.41494483 0.11844341 0.19506423
 0.13587654 0.11415239 0.43646343]
num_iter=4; x=[0.41132781 0.00887981 0.14496011 0.43483227 0.11448072 0.19946082
 0.13711446 0.10755014 0.44139385]
num_iter=5; x=[0.41985078 0.00091191 0.14193285 0.43730446 0.11396556 0.20011139
 0.13733721 0.10665769 0.44192814]
num_iter=6; x=[4.20732168e-01 9.14447449e-05 1.41617665e-01 4.37558722e-01
 1.13912574e-01 2.00179184e-01 1.37360977e-01 1.06564961e-01
 4.41982304e-01]
num_iter=7; x=[4.20820614e-01 9.14667549e-06 1.41586015e-01 4.37584224e-01
 1.13907259e-01 2.00185992e-01 1.37363368e-01 1.06555652e-01
 4.41987729e-01

In [20]:
sol[:n]

array([4.20829461e-01, 9.14751910e-07, 1.41582849e-01, 4.37586774e-01])

In [21]:
sol[:n].sum()

0.9999999983475308

In [22]:
sol[-m:]

array([0.11390673, 0.20018667, 0.13736361, 0.10655472, 0.44198827])

In [23]:
P @ sol[:n]

array([0.11390673, 0.20018667, 0.13736361, 0.10655472, 0.44198827])

In [19]:
func(sol, c_t, n, m)

-0.33189265089215736

In [13]:
alt = [ 0.40206418,  0.35333899, 0.04970143,  0.19489541]
alt2 = P @ alt

alt3 = np.hstack([alt, alt2])

In [14]:
func(alt3, c_t, n, m)

-0.21456748792968705